<h1 align = 'center'>
    <b>
        Machine Learning Practice - CS2008P
        <br>
        [21F1001893 - Humanshu D G]
    </b>
</h1>

Libraries allowed, only following libraries will be allowed to be used in the competition:
- Scikit-learn
- XGBoost
- imblearn
- NumPy
- Pandas
- SciPy
- Pickle
- Seaborn
- Matplotlib

## 0.1 Standard Kaggle Imports

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv
/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv


## 0.2 Importing Libraries

In [2]:
import xgboost as xgb, scipy as sp, matplotlib as mpl, seaborn as sns

from imblearn import under_sampling, over_sampling

%matplotlib inline

# **1. Get the Data**

## 1.1 Importing Dataset

In [3]:
test_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/test_data_v2.csv")
train_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/train_data_v2.csv")
sample_data = pd.read_csv("/kaggle/input/e-commerce-shoppers-behaviour-understanding/sample.csv")

train_data.shape, test_data.shape

((14731, 22), (6599, 21))

# **2. Preprocessing**

## 2.2 Data Imputation

In [4]:
train_data.isnull().sum()

HomePage                           153
HomePage_Duration                  150
LandingPage                        153
LandingPage_Duration               135
ProductDescriptionPage             123
ProductDescriptionPage_Duration    167
GoogleMetric:Bounce Rates          151
GoogleMetric:Exit Rates            129
GoogleMetric:Page Values           132
SeasonalPurchase                   150
Month_SeasonalPurchase             144
OS                                 134
SearchEngine                       122
Zone                               117
Type of Traffic                    143
CustomerType                       144
Gender                             145
Cookies Setting                    144
Education                          136
Marital Status                     130
WeekendPurchase                    121
Made_Purchase                        0
dtype: int64

## 2.2.1 Preprocessing Pipeline

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, QuantileTransformer, OrdinalEncoder

In [6]:
numerical_features = train_data.select_dtypes(include = ['int64', 'float64']).columns.tolist()
categorical_features = train_data.select_dtypes(include = ['object']).columns.tolist()

num_estimators = [
    ('simple_imputer', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),
    ('standard_scaler', StandardScaler()),
    ('quantile_transformer', QuantileTransformer())
]
num_pipeline = Pipeline(steps = num_estimators)

cat_estimators = [
    ('one_hot_encoder', OneHotEncoder())
]
cat_pipeline = Pipeline(steps = cat_estimators)


preprocessing_pipe = ColumnTransformer([
    ('cat_pipeline', cat_pipeline, categorical_features),
    ('num_pipeline', num_pipeline, numerical_features)
    
])

In [7]:
from sklearn import set_config

# displays HTML representation in a jupyter context
set_config(display = 'diagram')

preprocessing_pipe

ColumnTransformer(transformers=[('cat_pipeline',
                                 Pipeline(steps=[('one_hot_encoder',
                                                  OneHotEncoder())]),
                                 ['Month_SeasonalPurchase', 'CustomerType',
                                  'Gender', 'Cookies Setting', 'Education',
                                  'Marital Status']),
                                ('num_pipeline',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('standard_scaler',
                                                  StandardScaler()),
                                                 ('quantile_transformer',
                                                  QuantileTransformer())]),
                                 ['HomePage', 'HomePage_Duration',
                                  'LandingPage', 'LandingPage_Duration',
                                  'ProductDescriptionPage',
                                  'ProductDescriptionPage_Duration',
                                  'GoogleMetric:Bounce Rates',
                                  'GoogleMetric:Exit Rates',
                                  'GoogleMetric:Page Values',
                                  'SeasonalPurchase', 'OS', 'SearchEngine',
                                  'Zone', 'Type of Traffic',
                                  'WeekendPurchase'])])

### 2.2.1 Data Imputation on `train_data`

In [8]:
train_data.replace('nan', np.nan, inplace = True)

num = train_data.select_dtypes(include = ['int64', 'float64']).columns.tolist()
cat = train_data.select_dtypes(include = ['object']).columns.tolist()

# si_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')
# train_data[num] = si_mean.fit_transform(train_data[num])

# si_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
# train_data[cat] = si_mode.fit_transform(train_data[cat])

oe = OrdinalEncoder()
train_data[cat] = oe.fit_transform(train_data[cat])

knn = KNNImputer(n_neighbors = 10)
train_data[num] = knn.fit_transform(train_data[num])
train_data[cat] = knn.fit_transform(train_data[cat])

# ohe = OneHotEncoder(handle_unknown = 'ignore')
# enc_train_data = ohe.fit_transform(train_data[cat])
# enc_train_data_df = pd.DataFrame(enc_train_data.toarray(), columns = oe.get_feature_names_out(cat))
# train_data = pd.concat([train_data, enc_train_data_df], axis = 1)
# train_data.drop(cat, axis = 1, inplace = True)

ss = StandardScaler()
train_data[num] = ss.fit_transform(train_data[num])

qt = QuantileTransformer(output_distribution = 'uniform')
train_data[num] = qt.fit_transform(train_data[num])
# train_data.head()

### 2.2.1 (Alternative) Data Imputation on `train_data`

In [9]:
# X = preprocessing_pipe.fit_transform(X)
# X = pd.DataFrame(X)
# X.head()

### 2.2.2 Data Imputation on `test_data`

In [10]:
test_data.replace('nan', np.nan, inplace = True)

num = test_data.select_dtypes(include = ['int64', 'float64']).columns.tolist()
cat = test_data.select_dtypes(include = ['object']).columns.tolist()

# si_mean = SimpleImputer(missing_values = np.nan, strategy = 'mean')
# test_data[num] = si_mean.fit_transform(test_data[num])

# si_mode = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
# test_data[cat] = si_mode.fit_transform(test_data[cat])

oe = OrdinalEncoder()
test_data[cat] = oe.fit_transform(test_data[cat])

knn = KNNImputer(n_neighbors = 10)
test_data[num] = knn.fit_transform(test_data[num])
test_data[cat] = knn.fit_transform(test_data[cat])

# ohe = OneHotEncoder(handle_unknown = 'ignore')
# enc_test_data = ohe.fit_transform(test_data[cat])
# enc_test_data_df = pd.DataFrame(enc_test_data.toarray(), columns = oe.get_feature_names_out(cat))
# test_data = pd.concat([test_data, enc_test_data_df], axis = 1)
# test_data.drop(cat, axis = 1, inplace = True)

ss = StandardScaler()
test_data[num] = ss.fit_transform(test_data[num])

qt = QuantileTransformer(output_distribution = 'uniform')
test_data[num] = qt.fit_transform(test_data[num])
# test_data.head()

### 2.2.2 (Alternative) Data Imputation on `test_data`

In [11]:
# test_data = preprocessing_pipe.fit_transform(test_data)
# test_data = pd.DataFrame(test_data)
# test_data.head()

## Balancing using `imblearn`

In [12]:
# from imblearn.over_sampling import SMOTE

# X, y = SMOTE().fit_resample(X, y)
# X.shape, y.shape

## Outlier Adjustment

In [13]:
from scipy import stats

train_data = train_data[(np.abs(stats.zscore(train_data[num])) < 3).all(axis=1)]
# test_data = test_data[(np.abs(stats.zscore(test_data[num])) < 3).all(axis=1)]

In [14]:
train_data.shape, test_data.shape

((14540, 22), (6599, 21))

## Dropping Duplicates

In [15]:
print(train_data.duplicated().sum())
train_data = train_data.drop_duplicates()

299


In [16]:
train_data.shape, test_data.shape

((14241, 22), (6599, 21))

In [17]:
# mpl.pyplot.figure(figsize = (40, 40))
# sns.heatmap(train_data.corr(), annot = True, square = True, cmap='RdBu');

## 2.3 Splitting the Data for training

## 2.1 Separating features and labels

In [18]:
y, X = train_data.pop('Made_Purchase'), train_data

X.shape, y.shape

((14241, 21), (14241,))

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1127)

In [20]:
# X_train.describe().T

# 3. Baseline Model

In [21]:
# from sklearn.dummy import DummyClassifier

# dummy_clf = DummyClassifier(strategy = 'most_frequent')
# dummy_clf.fit(X, y)
# DummyClassifier(strategy = 'most_frequent')
# dummy_clf.predict(X)

# 4. Candidate Algorithms

In [22]:
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

best_accuracy = 0.0
best_classifier = 0
best_pipeline = ''

## 4.1 Logistic Regression

In [23]:
# from sklearn.linear_model import LogisticRegression

# pipeline_lr = Pipeline(
#                         [
#                             ('lr_classifier', LogisticRegression(max_iter = 100000))
#                         ]
#                       )

In [24]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', LogisticRegression())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [LogisticRegression()],
#                         'classifier__max_iter': [10000, 100000],
#                         'classifier__penalty': ['l1', 'l2'],
#                         'classifier__C': np.logspace(0, 4, 10),
#                         'classifier__solver': ['newton-cg', 'saga', 'sag', 'liblinear']
#                     }
#                 ]



# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# LogisticRegression(C=2.7825594022071245, max_iter=10000, penalty='l1',
#                    solver='saga') 
# Accuracy: 0.6628222523744912 
# F1: 0.28694404591104733

# LogisticRegression(C=2.7825594022071245, max_iter=10000, penalty='l1',
#                    solver='saga') 
# Accuracy: 0.6441581519324745 
# F1: 0.24075829383886257

# LogisticRegression(C=2.7825594022071245, max_iter=10000, penalty='l1',
#                    solver='saga') 
# Accuracy: 0.6612846612846612 
# F1: 0.4197233914612147

In [25]:
# clf_lr = LogisticRegression(C = 2.7825594022071245, max_iter = 10000, penalty = 'l1', solver = 'saga')
# clf_lr.fit(X_train, y_train)
# y_pred = clf_lr.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_lr, '\nAccuracy:', clf_lr.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))

In [26]:
# score_ = cross_val_score(clf_lr, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

## 4.2 Decision Tree Classifier

In [27]:
from sklearn.tree import DecisionTreeClassifier

pipeline_dt = Pipeline(
                        [
                            ('dt_classifier', DecisionTreeClassifier())
                        ]
                      )

In [28]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', DecisionTreeClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [DecisionTreeClassifier()],
#                         'classifier__max_depth': [1, 2, 5, 10, 100]
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# DecisionTreeClassifier(max_depth=1, max_leaf_nodes=10) 
# Accuracy: 0.6609336609336609 
# F1: 0.4180722891566265

In [29]:
clf_dt = DecisionTreeClassifier(max_depth = 1, max_leaf_nodes = 10)
clf_dt.fit(X_train, y_train)
y_pred = clf_dt.predict(X_test)
y_pred = y_pred.astype(bool)
# print(clf_dt, '\nAccuracy:', clf_dt.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))


# DecisionTreeClassifier(max_depth=1) 
# Accuracy: 0.6486006219458018 
# F1: 0.3500410846343468

# DecisionTreeClassifier(max_depth=1, max_leaf_nodes=10) 
# Accuracy: 0.6609336609336609 
# F1: 0.4180722891566265

In [30]:
# score_ = cross_val_score(clf_dt, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

# # Cross Validation Score: 0.6619583968701385

## 4.3 Random Forest Classifier

In [31]:
# from sklearn.ensemble import RandomForestClassifier

# pipeline_rf = Pipeline(
#                         [
#                             ('rf_classifier', RandomForestClassifier(criterion='gini', n_estimators=100))
#                         ]
#                       )

In [32]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [RandomForestClassifier()],
#                         'classifier__n_estimators': [10, 100, 1000],
#                         'classifier__min_samples_leaf': [2, 5, 10, 15, 100],
#                         'classifier__max_leaf_nodes': [2, 5, 10, 20],
#                         'classifier__max_depth': [None, 1, 2, 5, 8, 15, 25, 30]
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# RandomForestClassifier(max_depth=30, max_leaf_nodes=20, min_samples_leaf=100,
#                        n_estimators=1000) 
# Accuracy: 0.6784260515603799
# F1: 0.35422343324250677

In [33]:
# clf_rf = RandomForestClassifier(max_depth = 30,
#                                 max_leaf_nodes = 20,
#                                 min_samples_leaf = 100,
#                                 n_estimators = 1000)
# clf_rf.fit(X_train, y_train)
# y_pred = clf_rf.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_rf, '\nAccuracy:', clf_rf.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))

# # RandomForestClassifier(max_depth=30, max_leaf_nodes=20, min_samples_leaf=100,
# #                        n_estimators=1000) 
# # Accuracy: 0.6534873389604621 
# # F1: 0.28702010968921393

# # RandomForestClassifier(max_depth=30, max_leaf_nodes=20, min_samples_leaf=100,
# #                        n_estimators=1000) 
# # Accuracy: 0.6595296595296596 
# # F1: 0.3618421052631579

In [34]:
# score_ = cross_val_score(clf_rf, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

# # Cross Validation Score: 0.6655923048464668
# # Cross Validation Score: 0.666170694515041

## 4.4 ADAboost Classifier

In [35]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier

pipeline_adaboost = Pipeline(
                        [
                            ('adaboost_classifier', AdaBoostClassifier())
                        ]
                      )

In [36]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', AdaBoostClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [AdaBoostClassifier()],
# #                         'classifier__estimator': [GaussianNB(), DecisionTreeClassifier(max_depth = 1)],
#                         'classifier__algorithm': ['SAMME', 'SAMME.R'],
#                         'classifier__n_estimators': [1000, 4000, 6000, 10000],
#                         'classifier__learning_rate': [0.01, 0.05, 0.1, 0.5]
#                     }
#                 ]

# # dict_keys(['memory', 'steps', 'verbose', 'classifier',
# #            'classifier__algorithm', 'classifier__base_estimator',
# #            'classifier__learning_rate', 'classifier__n_estimators', 'classifier__random_state'])

# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# AdaBoostClassifier(algorithm = 'SAMME',
#                    estimators = clf_dt,
#                    learning_rate = 0.05,
#                    n_estimators = 4000)
# Accuracy: 0.6811397557666214 
# F1: 0.4035532994923857

# AdaBoostClassifier(algorithm='SAMME',
#                    base_estimator=DecisionTreeClassifier(max_depth=1),
#                    learning_rate=0.05, n_estimators=6000) 
# Accuracy: 0.6534873389604621 
# F1: 0.32055749128919864

# AdaBoostClassifier(algorithm='SAMME',
#                    base_estimator=DecisionTreeClassifier(max_depth=1),
#                    learning_rate=0.05, n_estimators=6000) 
# Accuracy: 0.7192498621070049 
# F1: 0.6563133018230924

In [37]:
# clf_ada = AdaBoostClassifier(algorithm = 'SAMME',
#                              base_estimator = clf_dt,
#                              learning_rate = 0.2,
#                              n_estimators = 1400)
# clf_ada.fit(X_train, y_train)
# y_pred = clf_ada.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_ada, '\nAccuracy:', clf_ada.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))


# # AdaBoostClassifier(algorithm='SAMME',
# #                    base_estimator=DecisionTreeClassifier(max_depth=1),
# #                    learning_rate=0.05, n_estimators=8000) 
# # Accuracy: 0.6534873389604621 
# # F1: 0.3193717277486911

# # AdaBoostClassifier(algorithm='SAMME',
# #                    base_estimator=DecisionTreeClassifier(max_depth=1),
# #                    learning_rate=0.005, n_estimators=8000) 
# # Accuracy: 0.6521545979564638 
# # F1: 0.30646589902568644

# # AdaBoostClassifier(algorithm='SAMME', base_estimator=DecisionTreeClassifier(max_depth=1,
# #                    max_leaf_nodes=10), learning_rate=0.05, n_estimators=8000) 
# # Accuracy: 0.6633906633906634 
# # F1: 0.3824855119124276

In [38]:
clf_ada = AdaBoostClassifier(algorithm = 'SAMME',
                             base_estimator = clf_dt,
                             learning_rate = 0.9,
                             n_estimators = 4000)
clf_ada.fit(X_train, y_train)
y_pred = clf_ada.predict(X_test)
y_pred = y_pred.astype(bool)
# print(clf_ada, '\nAccuracy:', clf_ada.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))

# AdaBoostClassifier(algorithm='SAMME',
#                    base_estimator=DecisionTreeClassifier(max_depth=1, max_leaf_nodes=10),
#                    learning_rate=0.09, n_estimators=4000) 
# Accuracy: 0.6633906633906634 
# F1: 0.3832797427652734

In [ ]:
score_ = cross_val_score(clf_ada, X_train, y_train, cv = 10).mean()
print('Cross Validation Score:', score_)

# Cross Validation Score: 0.6665930447650759
# Cross Validation Score: 0.6644157694499638

## 4.5 VotingClassifier

In [260]:
# from sklearn.ensemble import VotingClassifier

# pipeline_vc = VotingClassifier(
#                         [
#                             ('vc_classifier', VotingClassifier(
#                                                                 estimators = [
#                                                                                 ('ada', clf_ada),
#                                                                                 ('gnb', GaussianNB())
#                                                                             ]
#                                                                 )
#                             )
#                         ]
#                       )

In [186]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', VotingClassifier(estimators = [
#                                                                 ('ada', AdaBoostClassifier()),
#                                                                 ('gnb', GaussianNB())
#                                                             ]))])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [VotingClassifier(estimators = [
#                                                                 ('ada', AdaBoostClassifier()),
#                                                                 ('gnb', GaussianNB())
#                                                             ])],
#                         'classifier__voting': ['hard', 'soft']
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# VotingClassifier(estimators=[('ada', AdaBoostClassifier()),
#                              ('gnb', GaussianNB())]) 
# Accuracy 0.664179104477612 
# F1 0.32653061224489793

In [261]:
# clf_vc = VotingClassifier(estimators=[('ada', clf_ada), ('gnb', GaussianNB())])
# clf_vc.fit(X_train, y_train)
# y_pred = clf_vc.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_vc, '\nAccuracy', clf_vc.score(X_test, y_test), '\nF1', f1_score(y_test, y_pred))

VotingClassifier(estimators=[('ada',
                              AdaBoostClassifier(algorithm='SAMME',
                                                 base_estimator=DecisionTreeClassifier(max_depth=1,
                                                                                       max_leaf_nodes=10),
                                                 learning_rate=0.9,
                                                 n_estimators=4000)),
                             ('gnb', GaussianNB())]) 
Accuracy 0.663039663039663 
F1 0.415347137637028


In [41]:
# score_ = cross_val_score(clf_vc, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

Cross Validation Score: 0.6651285699154582


## 4.6 SVM Classifier

In [42]:
# from sklearn.svm import SVC

# pipeline_svm = Pipeline(
#                         [
#                             ('svm_classifier', SVC(gamma = 'auto'))
#                         ]
#                       )


# # kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’} 

In [43]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', SVC())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [SVC()],
#                         'classifier__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))

# SVC() 
# Accuracy 0.6648575305291723 
# F1 0.31955922865013775

In [44]:
# clf_svc = SVC()
# clf_svc.fit(X_train, y_train)
# y_pred = clf_svc.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_svc, '\nAccuracy', clf_svc.score(X_test, y_test), '\nF1', f1_score(y_test, y_pred))

SVC() 
Accuracy 0.643859649122807 
F1 0.2644927536231884


In [45]:
# score_ = cross_val_score(clf_svc, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

Cross Validation Score: 0.663925077512662


## 4.7 KNN Classifier

In [46]:
# from sklearn.ensemble import BaggingClassifier
# from sklearn.neighbors import KNeighborsClassifier

# pipeline_knn = Pipeline(
#                         [
#                             ('knn_classifier',
#                              BaggingClassifier(KNeighborsClassifier(),
#                                                max_samples = 0.5,
#                                                max_features = 0.5))
#                         ]
#                       )

In [47]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', BaggingClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [BaggingClassifier()],
#                         'classifier__warm_start': [True, False]
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))

# BaggingClassifier(warm_start=True) 
# Accuracy 0.49525101763907736 
# F1 0.2927756653992396

In [48]:
# clf_knn = BaggingClassifier()
# BaggingClassifier(KNeighborsClassifier(), warm_start = True, max_samples = 5, max_features = 5)
# clf_knn.fit(X_train, y_train)
# y_pred = clf_knn.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_knn, '\nAccuracy', clf_knn.score(X_test, y_test), '\nF1', f1_score(y_test, y_pred))

BaggingClassifier() 
Accuracy 0.5021929824561403 
F1 0.2766093052899936


In [49]:
# score_ = cross_val_score(clf_knn, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

Cross Validation Score: 0.5090506865407207


## 4.8 Multi-Layer Perceptron

In [50]:
# from sklearn.neural_network import MLPClassifier


# pipeline_mlp = Pipeline(
#                         [
#                             ('mlp_classifier', MLPClassifier()
#                             )
#                         ]
#                       )

In [51]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', MLPClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [MLPClassifier()],
#                         'classifier__hidden_layer_sizes': [(100, ), (400, ), (600, )],
#                         'classifier__activation': ['tanh', 'relu'],
#                         'classifier__solver': ['sgd', 'adam'],
#                         'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
#                         'classifier__max_iter': [1000, 10000]
#                     }
#                 ]


# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))


# # MLPClassifier(activation='tanh', hidden_layer_sizes=(400,),
# #               learning_rate='adaptive', max_iter=1000, solver='sgd') 
# # Accuracy: 0.666214382632293 
# # F1: 0.3031161473087819

In [52]:
# clf_mlp = MLPClassifier(hidden_layer_sizes = (40,30,20,10),
#                     max_iter = 1000,
#                     solver = 'sgd',
#                     activation = 'tanh',
#                     learning_rate = 'adaptive')
# clf_mlp.fit(X_train, y_train)
# y_pred = clf_mlp.predict(X_test)
# y_pred = y_pred.astype(bool)
# print(clf_mlp, '\nAccuracy:', clf_mlp.score(X_test, y_test), '\nF1:', f1_score(y_test, y_pred))

MLPClassifier(activation='tanh', hidden_layer_sizes=(40, 30, 20, 10),
              learning_rate='adaptive', max_iter=1000, solver='sgd') 
Accuracy: 0.6437139049311417 
F1: 0.2504672897196262


In [53]:
# score_ = cross_val_score(clf_mlp, X_train, y_train, cv = 10).mean()
# print('Cross Validation Score:', score_)

Cross Validation Score: 0.6653704525835492


## 4.9 Gradient Boosting Classifier

## 4.10 HistGradientBoostingClassifier

In [47]:
from sklearn.ensemble import HistGradientBoostingClassifier

pipeline_hgb = Pipeline(
                        [
                            ('hgb_classifier', HistGradientBoostingClassifier())
                        ]
                      )

# loss='log_loss', *, learning_rate = 0.01, max_iter=100, max_leaf_nodes=31, max_depth=None, min_samples_leaf=20, l2_regularization=0.0, max_bins=255, categorical_features=None, monotonic_cst=None, interaction_cst=None, warm_start=False, early_stopping='auto', scoring='loss', validation_fraction=0.1, n_iter_no_change=10, tol=1e-07, verbose=0, random_state=None, class_weight=None

In [49]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', HistGradientBoostingClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [HistGradientBoostingClassifier()],
#                         'classifier__loss': ['log_loss', 'auto', 'binary_crossentropy', 'categorical_crossentropy'],
#                         'classifier__learning_rate': np.logspace(0, 4, 10),
#                         'classifier__max_iter': [1000, 10000],
#                         'classifier__max_depth': [None, 1, 2, 5, 8, 15, 25, 30],
#                         'classifier__min_samples_leaf': [None, 2, 5, 10, 15, 100]
#                     }
#                 ]

# # loss='log_loss', 
# # learning_rate = 0.01,
# # max_iter=100,
# # max_leaf_nodes=31,
# # max_depth=None,
# # min_samples_leaf=20,
# # l2_regularization=0.0,
# # max_bins=255,
# # categorical_features=None,
# # monotonic_cst=None,
# # interaction_cst=None,
# # warm_start=False,
# # early_stopping='auto',
# # scoring='loss',
# # validation_fraction=0.1,
# # n_iter_no_change=10,
# # tol=1e-07,
# # verbose=0,
# # class_weight=None

# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))



/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


KeyboardInterrupt: 

# **5. Training**

## 5.1 Consolidating Pipelines

In [52]:
# pipelines = [pipeline_lr, pipeline_dt, pipeline_rf, pipeline_adaboost, pipeline_mlp]

# pipes = {
#             0: 'Logistic Regression',
#             1: 'Decision Tree',
#             2: 'Random Forest',
#             3: 'ADAboost',
#             4: 'Multi-Layer Perceptron'
# }

## 5.2 Fitting Models

In [53]:
# for pipe in pipelines:
#     pipe.fit(X_train, y_train)

## 5.3 Scoring of Models

### 5.3.1 Accuracy

In [54]:
# for _, classifier in enumerate(pipelines):
#     print('{} test accuracy: {}'.format(pipes[_], classifier.score(X_test, y_test)))


# # Logistic Regression test accuracy: 0.6628222523744912
# # Decision Tree test accuracy: 0.4497964721845319
# # Random Forest test accuracy: 0.5210312075983717
# # ADAboost test accuracy: 0.6723202170963365
# # Multi-Layer Perceptron test accuracy: 0.6648575305291723

### 5.3.2 F1

In [55]:
# from sklearn.metrics import f1_score


# for _, classifier in enumerate(pipelines):
#     y_pred = classifier.predict(X_test)
#     y_pred = y_pred.astype(bool)
#     print('{} f1-score: {}'.format(pipes[_], f1_score(y_test, y_pred)))
    
    
# # Logistic Regression f1-score: 0.28694404591104733
# # Decision Tree f1-score: 0.2453531598513011
# # Random Forest f1-score: 0.28771228771228774
# # ADAboost f1-score: 0.3893805309734514

# **6. Hyperparameter Tuning**

## 6.1 Importing GridSearchCV

In [56]:
# from sklearn.model_selection import GridSearchCV

## 6.2 Creating a pipeline fot GridSearchCV

In [57]:
# # creating a pipeline object
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# # creating a dictionary with candidate learning algorithms and their hyperparameters
# grid_parameters = [
#                     {
#                         'classifier': [LogisticRegression()],
#                         'classifier__max_iter': [10000, 100000],
#                         'classifier__penalty': ['l1', 'l2'],
#                         'classifier__C': np.logspace(0, 4, 10),
#                         'classifier__solver': ['newton-cg', 'saga', 'sag', 'liblinear']
#                     },
#                     {
#                         'classifier': [RandomForestClassifier()],
#                         'classifier__n_estimators': [10, 100, 1000],
#                         'classifier__min_samples_leaf': [2, 5, 10, 15, 100],
#                         'classifier__max_leaf_nodes': [2, 5, 10, 20],
#                         'classifier__max_depth': [None, 1, 2, 5, 8, 15, 25, 30]
                        
#                     },
#                     {
#                         'classifier': [AdaBoostClassifier()],
#                         'classifier__n_estimators': [100, 1000],
#                         'classifier__learning_rate': [0.001, 0.01],
#                         'classifier__random_state': [1127]
#                     },
#                     {
#                         'classifier': [VotingClassifier()],
#                         'classifier__voting': ['hard', 'soft']
#                     },
#                     {
#                         'classifier': [MLPClassifier()],
#                         'classifier__hidden_layer_sizes': [(100, ), (1000,)],
#                         'classifier__activation': ['identity', 'logistic', 'tanh', 'relu'],
#                         'classifier__solver': ['lbfgs', 'sgd', 'adam'],
#                         'classifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
#                         'classifier__max_iter': [200, 1000, 10000]
#                     },
#                 ]



# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)


# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))

## 6.3 Creating GridSearchCV

In [58]:
# grid_search = GridSearchCV(pipe, grid_parameters, cv = 4, verbose = 0, n_jobs = -1)
# best_model = grid_search.fit(X_train, y_train)

## 6.4 Fitting best parameters

In [59]:
# print(best_model.best_estimator_)
# print("The mean accuracy of the model is: ", best_model.score(X_test, y_test))

# Pipeline(steps=[('classifier',
#                 RandomForestClassifier(max_depth=5, max_leaf_nodes=10,
#                 min_samples_leaf=15))])
# The mean accuracy of the model is:  0.6709633649932157

# Pipeline(steps=[('classifier',
#                 AdaBoostClassifier(learning_rate=0.01, n_estimators=1000,
#                 random_state=1127))])
# The mean accuracy of the model is:  0.6811397557666214

## 6.5 Calculating scores of best parameters

### 6.5.1 F1

In [60]:
# y_pred = best_model.predict(X_test)
# y_pred = y_pred.astype(bool)
# print('{} f1-score: {}'.format(pipes[_], f1_score(y_test, y_pred)))
# # Random Forest f1-score: 0.3081312410841655
# # ADAboost f1-score: 0.4035532994923857

### 6.5.2 Cross Validation

In [61]:
# from sklearn.model_selection import cross_val_score

# score_ = cross_val_score(best_model, X_train, y_train, cv = 3).mean()
# print('{} cross_validation-score: {}'.format(pipes[_], score_))

# # Random Forest cross_validation-score: 0.6551260431050521

# Submitting as CSV file

In [236]:
sub = pd.DataFrame(clf_ada.predict(test_data), columns=['Made_Purchase'])
sub.index.name = 'id'
sub.to_csv("submission.csv", encoding='UTF-8')

output = pd.read_csv("submission.csv")

In [237]:
# with open('/kaggle/working/submission.csv') as f:
#     ff = f.readlines()
#     print(*ff)